In [ ]:
###################################################
# Trabajando con mongoDB                          #
###################################################

from pymongo import MongoClient, collection
from bson.objectid import ObjectId
from pprint import pprint
import sys, json

###################################################
# Conectar con un servidor mongoDB                #
###################################################

#client = MongoClient("127.0.0.1", "27017")
#client = MongoClient("localhost", "27017")
#client = MongoClient("mongodb://USER:PASSWORD@<IP>:<PORT>/")
#client = MongoClient("mongodb://127.0.0.1:27017/")

###################################################
# Ejecutar comando - Mostrar estado del servidor  #
###################################################

# Creamos un objeto que almacenamos en la variable clientDB
# El objeto representa el cliente para trabajar con las bases de datos de mongoDB
# Se requiere una cadena de conexión
clientDB = MongoClient("mongodb://localhost:27017/")

# Nos posicionamos sobre una base de datos, en el ejemplo sobre la base de datos ADMIN
db = clientDB.admin

# Ejecutamos un comando utilizando la función COMMAND
# El comando serverStatus nos retorna el estado del servidor en formato JSON
result = db.command("serverStatus")

# Mostramos el resultado de la ejecución del comando
pprint(result)

###################################################
# Trabajar con bases de datos y sus colecciones   #
###################################################

# Mostrar el nombre de las bases de datos
print(clientDB.list_database_names())

# El listado de bases de datos es una LISTA de python que podemos recorrer con un FOR
for db in clientDB.list_database_names():
    print(f"Nombre: {db}")
    print(f"-> {clientDB[db].list_collection_names()}\n")

# Seleccionar una base de datos con la que vamos a trabajar
db = clientDB.northwind             # Sintaxis de Objeto
db2 = clientDB["northwind"]         # Sintaxis de Colección

# Mostrar las colecciones que tiene una base de datos
# Las colecciones son equivalente a las tablas en las bases de datos realcionales 
print(db.list_collection_names())
print(db2.list_collection_names())

# Seleccionar una colección con la que vamos a trabajar
# collection = clientDB["northwind"]["customers"]
# collection = clientDB.northwind.customers
# collection = db["customers"]
collection = db.customers           # Todas son formas de hacer lo mismo

# Mostramos el número de documentos
# Los documentos son equivalentes a los registros en bases de datos relacionales
print(f"La colección {collection.name} tiene {collection.estimated_document_count()} documentos.")

###################################################
# Trabajar con los documentos de las colecciones  #
###################################################

# Mostrar el documento por identificador del objeto
# Filtro: _id = 
result = collection.find_one({"_id": ObjectId("663a1062bcd71e98892bf2a6")})
pprint(result)
print("")

# Mostrar el primer documento que coincide con el filtro
# Filtro: Country = USA
result = collection.find_one({"Country": "USA"})
pprint(result)
print("")

# Mostrar todos los documentos que coinciden con el filtro
# Filtro: Country = USA
# Retorna un cursor
cursor = collection.find({"Country": "USA"})

# Mostrar el número de documentos de una búsqueda
# print(f"Resultado de la búsqueda son {cursor.count()} documentos")    # No disponible desde la versión ___
print(f"Resultado de la búsqueda son {collection.count_documents({"Country": "USA"})} documentos\n")

# Cuando ALIVE retorna TRUE significa que tenemos documentos pendientes de leer en el cursor
print(f"¿Quedan documentos pendientes de leer? {cursor.alive}\n")

# Utilizamos WHILE para mostrar los documentos del cursor
# el WHILE se ejecuta mientras ALIVE retorne TRUE
# Con la función .NEXT() para posicionarnos en el siguiente documento del cursor
while cursor.alive:
    pprint(cursor.next())
    print("")

print(f"¿Quedan documentos pendientes de leer? {cursor.alive}\n")


###################################################
# Ejemplos: Búsquedas, Utilización de operadores  #
###################################################
dbNorthwind = clientDB.northwind
coleccionClientes = dbNorthwind.customers
coleccionPedidos = dbNorthwind.orders
coleccionProductos = dbNorthwind.products
"""
===================================================
 Listado de operadores relacionales
===================================================
$eq     - equal - igual
$ne     - not equal - distinto

$lt     - low than - menor que
$lte    - low than equal - menor o igual que

$gt     - greater than - mayor que
$gte    - greater than equal - mayor o igual que

$in     - in - dentro de
$nin    - not in - no dentro de
$regex  - cumple con la expresión regular
"""

#clientesFiltrados = coleccionClientes.find({"Country": "USA"})
#clientesFiltrados = coleccionClientes.find({"Country": "USA"}).limit(3)
#clientesFiltrados = coleccionClientes.find({"Country": "USA"}).skip(5)
#clientesFiltrados = coleccionClientes.find({"Country": "USA"}).sort("City")      # Ordenados de A a W
#clientesFiltrados = coleccionClientes.find({"Country": "USA"}).sort({"City": 1}) # Ordenados de A a W
clientesFiltrados = coleccionClientes.find({"Country": "USA"}).sort({"City": -1}) # Ordenados de W a A

# Buscar clientes de USA, ejemplos con y sin operador
clientesFiltrados = coleccionClientes.find({"Country": "USA"})                    # Sin operador
clientesFiltrados = coleccionClientes.find({"Country": {"$eq": "USA"}})           # Con operador

# Buscar clientes fuera de USA
clientesFiltrados = coleccionClientes.find({"Country": {"$ne": "USA"}})

# Buscar clientes de USA y MExico, ordenados por país y ciudad
clientesFiltrados = coleccionClientes.find({"Country": {"$in": ["USA", "Mexico"]}}).sort([("Country", 1), ("City", 1)])

# Buscar clientes que contienen DE en la clave CustomerID
clientesFiltrados = coleccionClientes.find({"CustomerID": {"$regex": "DE"}})

# Buscar clientes que el CustomerID empieza por A y finaliza con 4 caracteres más 
clientesFiltrados = coleccionClientes.find({"CustomerID": {"$regex": "A[A-Z]{4}"}})

# Buscar clientes de la ciudad de San Francisco en USA 
clientesFiltrados = coleccionClientes.find({"Country": "USA", "City": "San Francisco"})

# Buscar clientes de la ciudad de San Francisco en USA utilizando el operador AND
clientesFiltrados = coleccionClientes.find({"$and": [{"Country": "USA"}, {"City": "San Francisco"}]})

# Buscar clientes de GERMANY o USA utilizando el operador OR
clientesFiltrados = coleccionClientes.find({"$or": [{"Country": "Germany"}, {"Country": "USA"}]})

# Buscar los clientes de Mexico y sus pedidos
clientesFiltrados = coleccionClientes.find({"Country": "Mexico"})

while clientesFiltrados.alive:
    cliente = clientesFiltrados.next()
    print(f"ID del Cliente: {cliente["CustomerID"]}# - Nombre de la Compañía: {cliente["CompanyName"]} - País: {cliente["Country"]} - Ciudad: {cliente["City"]}")

    pedidosFiltrados = coleccionPedidos.find({"CustomerID": cliente["CustomerID"]})
    while pedidosFiltrados.alive:
        pedido = pedidosFiltrados.next()
        print(f">>> ID del Pedido: {pedido["OrderID"]}# - Fecha del Pedido: {pedido["ShippedDate"]}")

    print("--------------------")

# Buscar los clientes de Mexico y sus pediddos utilizando la función AGGREGATE
clientesConPedidosFiltrados = coleccionClientes.aggregate([
    {"$match": {"Country": "Mexico"}},
    {"$sort": {"City": 1}},
    {"$lookup": {
        "from": "orders",
        "localField": "CustomerID",
        "foreignField": "CustomerID",
        "as": "Pedidos"
    }}
])

while clientesConPedidosFiltrados.alive:
    clienteConPedidos = clientesConPedidosFiltrados.next()
    print(f"ID del Cliente: {clienteConPedidos["CustomerID"]}# - Nombre de la Compañía: {clienteConPedidos["CompanyName"]} - País: {clienteConPedidos["Country"]} - Ciudad: {clienteConPedidos["City"]}")

    for pedido in clienteConPedidos["Pedidos"]:
        print(f">>> ID del Pedido: {pedido["OrderID"]}# - Fecha del Pedido: {pedido["ShippedDate"]}")
    
    print("")

# Buscamos todos los productos con UnitsInStock distinto de cero
# Convertir UnitInStock y UnitPrice en valores numéricos
# Calcular la suma de multiplicar el precio por unidades de cada producto

productosFiltrados = coleccionProductos.find({"UnitsInStock": {"$ne": 0}})

total = 0
while productosFiltrados.alive:
    producto = productosFiltrados.next()
    unidadesEnStock = int(producto["UnitsInStock"])
    precio = float(producto["UnitPrice"])
    total += unidadesEnStock * precio

print(f"Valor del Stock: {total:1.2f}")

# Utilizamos AGGREGATE para calcular el valor del stock
query = [
    {"$match": {"UnitsInStock": {"$ne": "0"}}},
    {"$addFields": {
        "Precio": {"$toDouble": "$UnitPrice"},
        "Unidades": {"$toInt": "$UnitsInStock"}
    }},
    {"$group": {
        "_id": "Valor del Stock",
        "Total": {"$sum": {"$multiply": ["$Precio", "$Unidades"]}},
        "Productos": {"$sum": 1}
    }}
]

productosFiltrados = coleccionProductos.aggregate(query)
pprint(productosFiltrados.next())


###################################################
# CRUD con documentos                             #
###################################################

# Creamos la clase Customer que define un cliente de nuestra base de datos
class Customer:
    CustomerID = None
    CompanyName = None
    ContactName = None
    ContactTitle = None
    Address = None
    City = None
    Region = None
    PostalCode = None
    Country = None
    Phone = None
    Fax = None
    
    def __init__(self, customerID, companyName, contactName, contactTitle, adress, city, region, postalCode, country, phone, fax) -> None:
        self.CustomerID = customerID
        self.CompanyName = companyName
        self.ContactName = contactName
        self.ContactTitle = contactTitle
        self.Address = adress
        self.City = city
        self.Region = region
        self.PostalCode = postalCode
        self.Country = country
        self.Phone = phone
        self.Fax = fax

###################################################
# Insertar documentos                             #
###################################################

# Insertamos partiendo de la clase Customer
cliente = Customer("DEMO1", "Empresa Uno, SL", "Alejandro", "Gerente", "Calle Uno, S/N", "Granada", "Granada", "18195", "España", "900100100", "900100200")

# Todos los objetos de python tienen una propiedad que es __dict__
# que retorna un diccionario de todas sis variables
pprint(cliente.__dict__)

id = coleccionClientes.insert_one(cliente.__dict__).inserted_id
print(f"ID del nuevo documento: {id}")

# Insertamos partiendo de un diccionario
cliente2 = {
    "CustomerID": "DEMO2",
    "CompanyName": "Empresa Dos, SL",
    "ContactName": "Borja Cabeza",
    "ContactTitle": "Gerente",
    "Address": "Calle Dos S/N",
    "City": "Madrid",
    "Region": "Madrid",
    "PostalCode": "28019",
    "Country": "España",
    "Phone": "910 101 102",
    "Fax": "910 101 103"}

id = coleccionClientes.insert_one(cliente2).inserted_id
print(f"ID del nuevo documento: {id}")

###################################################
# Actualizar documentos                           #
###################################################

query = {"CustomerID": "DEMO2"}
clienteFiltrado = coleccionClientes.find_one(query)
pprint(clienteFiltrado)

# Actualizamos uno o varios documentos de una colección

# Nuevos valores para el documento o documentos que vamos a actualizar
newValues = {"$set": {
    "ContactName": "Ana Sanz",
    "PostalCode": "28013"
}}

# Actualizar el primer documento que retorna la consulta
result = coleccionClientes.update_one(query, newValues)

# Actualizar todos los documentos que retorna la consulta
result = coleccionClientes.update_many(query, newValues)

print(f"{result.matched_count} documentos encontrados")
print(f"{result.modified_count} documentos modificados")
print(result)

pprint(coleccionClientes.find_one(query))

###################################################
# Eliminar documentos                             #
###################################################

# Eliminar el primer documento coincide con el filtro de búsqueda
result = coleccionClientes.delete_one({"CustomerID": "DEMO2"})
print(result)
print(f"{result.deleted_count} documentos eliminados.")

# Eliminar todos los documentos coincidentes con el filtro de búsqueda
result = coleccionClientes.delete_many({"CustomerID": "DEMO2"})
print(result)
print(f"{result.deleted_count} documentos eliminados.")

In [ ]:
###################################################
# Trabajando con Microsoft SQL Server             #
###################################################

import pymssql


# Establecer la conexión con la base de datos
connection = pymssql.connect(
    server="hostdb-eoi.database.windows.net",
    user="Administrador",
    password="azurePa$$w0rd",
    database="Northwind"
)

# Creamos un cursor para ejecutar comandos en la base de datos
# Creamos un cursor que retorna Tuplas
cursor = connection.cursor()

# Creamos un cursor que retorna Diccionarios
cursor = connection.cursor(as_dict=True)


###################################################
# SELECT, leer registros de la base de datos      #
###################################################

# Ejemplos del comando SELECT, para leer registros de la base de datos
cursor.execute("SELECT * FROM dbo.Customers")

# Mostrar el contenido del cursor utilizando un WHILE
row = cursor.fetchone()
while(row):
    print(f"ID:      {row["CustomerID"]}")
    print(f"Empresa: {row["CompanyName"]} - {row["City"]} ({row["Country"]})\n")
    row = cursor.fetchone()

# El siguiente ejemplo comentado muestra como tratar los registros cuando se 
# entregan Tuplas en lugar de Diccionarios
row = cursor.fetchone()
while(row):
    print(f"ID:      {row[0]}")
    print(f"Empresa: {row[1]} - {row[5]} ({row[8]})\n")
    row = cursor.fetchone()

# Mostrar el contenido del cursor utilizando un FOR
clientes = cursor.fetchall()

for cliente in clientes:
    print(f"ID:      {cliente["CustomerID"]}")
    print(f"Empresa: {cliente["CompanyName"]} - {cliente["City"]} ({cliente["Country"]})\n")


# Ejemplos del comando SELECT, para leer registros de la base de datos
cursor.execute("SELECT * FROM dbo.Customers")
cursor.execute("SELECT * FROM dbo.Customers WHERE Country = 'USA'")
cursor.execute("SELECT * FROM dbo.Customers WHERE Country = %d", "USA")

#pais = input("Nombre del país: ")
#cursor.execute(f"SELECT * FROM dbo.Customers WHERE Country = '{pais}'")
#cursor.execute("SELECT * FROM dbo.Customers WHERE Country = %d", pais)

cursor.execute("SELECT * FROM dbo.Customers WHERE Country = 'USA' AND City = 'San Francisco'")
cursor.execute("SELECT * FROM dbo.Customers WHERE Country = 'USA' OR Country = 'Germany'")

cursor.execute("SELECT CompanyName, CustomerID, City, Country FROM dbo.Customers WHERE Country = 'USA'")

cursor.execute("SELECT * FROM dbo.Customers WHERE Country = 'USA' OR Country = 'Germany' ORDER BY Country")
cursor.execute("SELECT * FROM dbo.Customers WHERE Country = 'USA' OR Country = 'Germany' ORDER BY Country ASC")
cursor.execute("SELECT * FROM dbo.Customers WHERE Country = 'USA' OR Country = 'Germany' ORDER BY Country DESC")
cursor.execute("SELECT * FROM dbo.Customers WHERE Country = 'USA' OR Country = 'Germany' ORDER BY Country, City")


clientes = cursor.fetchall()
for cliente in clientes:
    print(f"ID:      {cliente["CustomerID"]}")
    print(f"Empresa: {cliente["CompanyName"]} - {cliente["City"]} ({cliente["Country"]})\n")



cursor = connection.cursor(as_dict=True)

# Ejemplo que comienza con una consulta y realiza 830 subconsultas dentro del FOR
cursor.execute("SELECT * FROM dbo.Orders")

for row in cursor.fetchall():
    print(f"-> {row["OrderID"]}# - {row["CustomerID"]} {row["OrderDate"]}")

    cursor2 = connection.cursor(as_dict=True)
    #cursor2.execute("SELECT * FROM dbo.Employees WHERE EmployeeID = %d", row["EmployeeID"])     # Son lo mismmo
    cursor2.execute(f"SELECT * FROM dbo.Employees WHERE EmployeeID = {row["EmployeeID"]}")
    employee = cursor2.fetchone()

    print(f"   Pedido gestionado por {row["EmployeeID"]}: {employee["FirstName"]} {employee["LastName"]}")


# Ejemplo que se realiza con una sola consulta

#cursor.execute("SELECT * FROM dbo.Orders AS o, dbo.Employees AS e")                             # Es lo mismo
#cursor.execute("SELECT * FROM dbo.Orders o, dbo.Employees e")

cursor.execute("SELECT o.OrderID, o.CustomerID, o.OrderDate, o.EmployeeID, e.FirstName, e.LastName FROM dbo.Orders o, dbo.Employees e WHERE o.EmployeeID = e.EmployeeID")

for row in cursor.fetchall():
    print(f"-> {row["OrderID"]}# - {row["CustomerID"]} {row["OrderDate"]}")
    print(f"   Pedido gestionado por {row["EmployeeID"]}: {row["FirstName"]} {row["LastName"]}")

###################################################
# SELECT, ejemplos de JOIN                        #
###################################################

query = """
    SELECT o.OrderID, o.CustomerID, o.OrderDate, o.EmployeeID, e.FirstName, e.LastName 
    FROM dbo.Orders o
    JOIN dbo.Employees e
    ON o.EmployeeID = e.EmployeeID
"""
cursor.execute(query)

for row in cursor.fetchall():
    print(f"-> {row["OrderID"]}# - {row["CustomerID"]} {row["OrderDate"]}")
    print(f"   Pedido gestionado por {row["EmployeeID"]}: {row["FirstName"]} {row["LastName"]}")


###################################################
# SELECT, las agrupaciones                        #
###################################################

# Listado de clientes de USA y el número de pedidos de cada cliente
# NO ES UNA AGRUPACIÓN, es una subconsulta dentro del FOR que DEBEMOS evitar por rendimiento
query = "SELECT * FROM dbo.Customers WHERE Country = 'USA'"
cursor.execute(query)

for row in cursor.fetchall():
    cursor2 = connection.cursor(as_dict=True)
    cursor2.execute(f"SELECT COUNT(*) FROM dbo.Orders WHERE CustomerID = '{row["CustomerID"]}'")
    print(f"{row["CustomerID"]}# {row["CompanyName"]} -> {cursor2.fetchone()[0]} pedidos")

"""
    Opcionalmente podemos trabajar con cursores que retornan diccionarios, pero estamos
    obligados a definir alias para el dato calculado usando AS

    cursor2 = connection.cursor(as_dict=True)
    cursor2.execute(f"SELECT COUNT(*) AS numPedidos FROM dbo.Orders WHERE CustomerID = '{row["CustomerID"]}'")
    print(f"{row["CustomerID"]}# {row["CompanyName"]} -> {cursor2.fetchone()["numPedidos]} pedidos")    
"""

# Listado de clientes de USA y el número de pedidos de cada cliente
# Solo los clientes con más de 10 pedidos
query = """
    SELECT c.CustomerID, c.CompanyName, COUNT(o.OrderID) as NumPedidos
    FROM dbo.Customers c
    JOIN dbo.Orders o
    ON c.CustomerID = o.CustomerID
    WHERE c.Country = 'USA'
    GROUP BY c.CustomerID, c.CompanyName
    HAVING COUNT(o.OrderID) > 10
"""
cursor.execute(query)

for row in cursor.fetchall():
    print(f"-> {row["CustomerID"]}# - {row["CompanyName"]} -> {row["NumPedidos"]} pedidos")


######################################################
# INSERT, insertar nuevo registros
######################################################

# Definición de un objeto que representa el registro CUSTOMER 
class Customer:
    CustomerID = None
    CompanyName = None
    ContactName = None
    ContactTitle = None
    Address = None
    City = None
    Region = None
    PostalCode = None
    Country = None
    Phone = None
    Fax = None

# Instanciamos el objeto CUSTOMER
cliente = Customer()
cliente.CustomerID = "DEMO1"
cliente.CompanyName = "Empresa Uno, SL"
cliente.ContactName = "Borja"
cliente.ContactTitle = "Gerente"
cliente.Address = "Calle Uno, S/N"
cliente.City = "Madrid"
cliente.Region = "Madrid"
cliente.PostalCode = "28016"
cliente.Country = "España"
cliente.Phone = "900100100"
cliente.Fax = "900100200"

# cliente2 es un diccionario que también representa el registro CUSTOMER
cliente2 = {"CustomerID": "DEMO2",
            "CompanyName": "Empresa Dos, SL",
            "ContactName": "Borja Cabeza",
            "ContactTitle": "Gerente",
            "Address": "Calle Dos S/N",
            "City": "Madrid",
            "Region": "Madrid",
            "PostalCode": "28019",
            "Country": "España",
            "Phone": "910 101 102",
            "Fax": "910 101 103"}

# INSERT comando de inserción
command = """
    INSERT INTO dbo.Customers(CustomerID, CompanyName, ContactTitle, City, Country)
    VALUES('BCR01', 'Company SL', 'Borja Cabeza', 'Madrid', 'España')
"""

# Insertamos nuevos registros ejecutado el comando INSERT
cursor.execute(command)

# Utilizamos la función commit() de la conexión para CONFIRMAR la transacción
# tanto para operaciones de inserción, actualización y borrado
connection.commit()

# Utilizamos la función rollback() de la conexión para ANULAR la transacción
# tanto para operaciones de inserción, actualización y borrado
connection.rollback()

# Ejemplo de un comando INSERT que indica las columnas o campos y sus valores
command = """
    INSERT INTO dbo.Customers(
        CustomerID, 
        CompanyName, 
        ContactTitle, 
        City, 
        Country) VALUES('BCR01', 'Company SL', 'Borja Cabeza', 'Madrid', 'España')
"""
# Ejemplo de un comando INSERT que indica las columnas o campos y comodines para los valores
command2 = """
    INSERT INTO dbo.Customers(
        CustomerID, 
        CompanyName, 
        ContactName,
        ContactTitle, 
        City, 
        Country) VALUES(%s, %s, %s, %s, %s, %s)
"""

# Al ejecutar el comando con comodines, pasamos como segundo parámetro los valores en una lista
cursor.execute(command2, ["BCR02", "Company Demo, SL", "Borja", "CEO", "Valencia", "España"])

# El mismo ejemplo donde pasamos los valores en una tupla
cursor.execute(command2, ("BCR03", "Company Demo, SL", "Borja", "CEO", "Valencia", "España"))
connection.commit()

# Para insertar varios registros al mismo tiempo creamos una lista que contiene en cada posición
# una tupla con los valores de cada registro que vamos a insertar
data = []
data.append(("BCR10", "Company Demo 10, SL", "Borja", "CEO", "Sevilla", "España"))
data.append(("BCR11", "Company Demo 11, SL", "Carlos", "CEO", "Bilbao", "España"))
data.append(("BCR12", "Company Demo 12, SL", "Julian", "CEO", "Málaga", "España"))    

# Utilizamos la función .executemany() para insertar varios registros y pasamos como segundo
# parámetro la lista de tuplas con los valores de los diferentes registros
cursor.executemany(command2, data)
connection.commit()

# La propiedad o variable .rowcount nos devuelve el número de registros insertados, actualizados o borrados
print(f"{cursor.rowcount} registros insertados.")

# Ejemplo de un INSERT donde se especifican valores para todos los campos o columnas del registro
command = """
    INSERT INTO dbo.Customers VALUES(
        'DEMO2',
        'Empresa Dos, SL',
        'Borja Cabeza',
        'Gerente',
        'Calle Dos S/N',
        'Madrid',
        'Madrid',
        '28019',
        'España',
        '910 101 102',
        '910 101 103')
"""

# Ejemplo de un INSERT donde se especifican comodines para los valores para todos los 
# campos o columnas del registro
command = """
    INSERT INTO dbo.Customers VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""


######################################################
# UPDATE, actulizar registros
######################################################

command = """
    UPDATE dbo.Customers
    SET Address = 'Calle Uno, S/N', ContactName = 'Carlos Sánchez'
    WHERE CustomerID = 'BCR11'
"""

# cursor.execute(command)
connection.commit()

print(f"{cursor.rowcount} registros actualizados.")


command = """
    UPDATE dbo.Customers
    SET Address = %s, ContactName = %s
    WHERE CustomerID = 'BCR12'
"""

cursor.execute(command, ("Calle Principal, 10", "María José Sanz"))
connection.commit()

print(f"{cursor.rowcount} registros actualizados.")


command = """
    UPDATE dbo.Customers
    SET Address = %s, ContactName = %s
    WHERE CustomerID = %s
"""

cursor.execute(command, ("Calle Principal, 10", "María Sanz", "BCR12"))
connection.commit()

print(f"{cursor.rowcount} registros actualizados.")


######################################################
# DELETE, eliminar registros
######################################################

command = """
    DELETE FROM dbo.Customers
    WHERE CustomerID = 'BCR10'
"""

try:
    cursor.execute(command)
    connection.commit()
except Exception as e:
    connection.rollback()
    print(f"Error: {e}")
finally:
    print(f"{cursor.rowcount} registros eliminados.")
    connection.close()


command = """
    DELETE FROM dbo.Customers
    WHERE CustomerID = %s
"""

try:
    cursor.execute(command, ("BCR11"))
    connection.commit()
except Exception as e:
    connection.rollback()
    print(f"Error: {e}")
finally:
    print(f"{cursor.rowcount} registros eliminados.")
    connection.close()